In [1]:
""" Fit Discrete Distributions to Data"""
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [2]:
# Read the data from Excel file 
data = pd.read_excel ("7.4.3_data.xlsx", sheet_name="data2", header=None)
data = data.values.flatten()

In [3]:
# Plot data and show description
fig0, ax0 = plt.subplots(dpi=100)
values, counts = np.unique(data, return_counts=True)
ax0.bar(values, counts/counts.sum())
ax0.set_xticks(np.arange(values.min()-1, values.max()+2))
ax0.set_xlabel("Data value")
ax0.set_ylabel("Frequency density")
ax0.grid('dotted')
plt.show()

# Display description of the data
desc = stats.describe(data)
print("\nData Description:")
print(f"  size = {desc.nobs}")
print(f"  minmax = {desc.minmax}")
print(f"  mean = {desc.mean}")
print(f"  var = {desc.variance}")
print(f"  skewness = {desc.skewness}")
print(f"  kurtosis= {desc.kurtosis}")


Data Description:
  size = 100
  minmax = (3, 17)
  mean = 8.18
  var = 8.492525252525253
  skewness = 0.5715704116345816
  kurtosis= 0.0660478548040615


In [4]:
# Distributions to fit and their parameter initial guess
#  'poisson'   # Poisson (mu)
#  'binom'     # Binomial (n, p)
#  'nbinom'    # Negative Binomial (n, p)
#  'betabinom' # Beta-Binomial (n, a, b)
#  'randint'   # randint (a, b)
#  'geom'      # geom(p)

Dists = { 'poisson'  : (10, ),
          'binom'    : (20, 0.5),
          'nbinom'   : (20, 0.5),
          'betabinom': (20, 10, 10),
          'randint'  : (3, 17),
          'geom'     : (0.5,)}

In [5]:
# Negative Log Likelihood function to minimize
def nLogLikelihood(params, dist, data):
    return -dist.logpmf(data, *params).sum() 

In [6]:
# Dict to keep successful results as we loop through the distributions
res = {}
for d, x0 in Dists.items():
    dist = getattr(stats, d)
    # Perform MLE Optimization
    sol = minimize(nLogLikelihood, x0=x0, args=(dist, data),
      method='Nelder-Mead',
      options={'maxiter':50000,'xatol':1E-8,'fatol':1E-8})
    # method='Powell',
    # options={'maxiter':10000,'xtol':1E-8,'ftol':1E-8})
    print(f"\nFitting Distribution {d}:")
    print(f"  MLE {sol.message}")
    params_fit = sol.x
    print(f"  Fitted Parameters = {params_fit}")
    # Keep the results if successful
    if sol.success:
        # Perform Kolmogorov-Smirnov test and store result
        Dv, pv = stats.kstest(data, d, args=params_fit)
        res[d] = {'Params' : params_fit,
                  'KS_D' : Dv,
                  'KS_pv': pv }

print(f"\nNumber of distributions fitted = {len(res)}")

# Sort the results by KS_stats
results = {dist: vals for dist, vals in 
        sorted(res.items(), key=lambda x: x[1]['KS_D']) }


Fitting Distribution poisson:
  MLE Optimization terminated successfully.
  Fitted Parameters = [8.17999996]

Fitting Distribution binom:
  MLE Optimization terminated successfully.
  Fitted Parameters = [33.       0.24375]

Fitting Distribution nbinom:
  MLE Optimization terminated successfully.
  Fitted Parameters = [308.57602061   0.97417571]

Fitting Distribution betabinom:
  MLE Optimization terminated successfully.
  Fitted Parameters = [18.  10.5 10.5]


C:\Users\isepohkl\AppData\Local\anaconda3\lib\site-packages\scipy\optimize\_optimize.py:863: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):



Fitting Distribution randint:
  MLE Maximum number of iterations has been exceeded.
  Fitted Parameters = [ 3. 17.]

Fitting Distribution geom:
  MLE Optimization terminated successfully.
  Fitted Parameters = [0.12224939]

Number of distributions fitted = 5


In [7]:
# Plot PMF of fitted distributions and compare them with the data
num_to_plot = 5
num_to_plot = min(num_to_plot, len(results))

fig1, ax1 = plt.subplots(dpi=100)
ax1.bar(values, counts/counts.sum(), label='data', alpha=0.4)
ax1.set_ylabel("Probability")
x = np.arange(values.min()-1, values.max()+2)
for d in list(results.keys())[:num_to_plot]:
    dist = getattr(stats, d)
    params_fit = results[d]['Params']
    ax1.scatter(x, dist.pmf(x, *params_fit), label=d)
ax1.set_xticks(x)
ax1.legend()
ax1.grid(ls='--')
plt.show()

In [8]:
# Plot CDF of fitted distributions and compare them with the data
def step_func(x, y):
    ''' Compute coordinates to line plot x and y as step function '''
    xval = []
    for item in x:
        xval.append(item)
        xval.append(item)
    yval = [0]
    for item in y[:-1]:
        yval.append(item)
        yval.append(item)
    yval.append(1)
    return (xval, yval)
    
fig2, ax2 = plt.subplots(dpi=100)
cdf_data = np.cumsum(counts/counts.sum())
ax2.plot(*step_func(values, cdf_data), label='data' )
ax2.scatter(values , cdf_data, label=None)
x = np.arange(values.min()-1, values.max()+2)
for d in list(results.keys())[:num_to_plot]:
    dist = getattr(stats, d)
    params_fit = results[d]['Params']
    ax2.scatter(x, dist.cdf(x, *params_fit), label=d)
ax2.set_xticks(x)
ax2.set_yticks(np.linspace(0,1,11))
ax2.set_ylabel('CDF')
ax2.legend()
ax2.grid(ls='--')
plt.show()

In [9]:
# Show the top fitted parameters
num_to_show = 5
num_to_show = min(num_to_show, len(results))
print(f"\nThe top {num_to_show} distributions:")
  
for i, (d, vals) in enumerate(list(results.items())[:num_to_show]):
    dist = getattr(stats, d)
    params = vals['Params']
    print(f"\nDistribution {i+1}: {d}")
    print(f"  Params = {vals['Params']}")
    print(f"  KS_stats = {vals['KS_D']}")
    pv = vals['KS_pv']
    print(f"  p-value = {pv}", end=" ")
    if pv < 0.05:
        print(" < 0.05")
    else:
        print("")
    print(f"  mean = {dist(*params).mean()}")
    print(f"  var = {dist(*params).var()}")
    print(f"  sd = {dist(*params).std()}")


The top 6 distributions:

Distribution 1: poisson
  Params = [8.17999996]
  KS_stats = 0.13188736268091658
  p-value = 0.05614609245552149 
  mean = 8.179999962449074
  var = 8.179999962449074
  sd = 2.8600699226503314

Distribution 2: nbinom
  Params = [308.57602061   0.97417571]
  KS_stats = 0.13520999427877164
  p-value = 0.04686151753816992  < 0.05
  mean = 8.179999455336235
  var = 8.396841940232385
  sd = 2.897730480950978

Distribution 3: binom
  Params = [33.       0.24375]
  KS_stats = 0.150837808663566
  p-value = 0.018845645860325555  < 0.05
  mean = 8.043750000000006
  var = 6.0830859375000035
  sd = 2.466391278264664

Distribution 4: betabinom
  Params = [18.  10.5 10.5]
  KS_stats = 0.17607889576275854
  p-value = 0.003498486645988952  < 0.05
  mean = 9.0
  var = 7.9772727272727275
  sd = 2.824406615073815

Distribution 5: geom
  Params = [0.12224939]
  KS_stats = 0.3889784302638508
  p-value = 3.9009361387286217e-14  < 0.05
  mean = 8.18000000390055
  var = 58.732400059